# Notebook #1: ASTROMETRY.

This notebook reproduces the astrometrical analysis based on Gaia data in order to determine the local movement and space velocity of LS 2355 compared to its surroundings.

We take the following steps:

0) import the necessary packages
1) define the local ISM motion correction
2) show an example calculation using pre-Gaia constraints
3) perform the Gaia-based calculation #1: Oort constants from Comeron & Pasquali (2007) and no radial motion
4) perform the Gaia-based calculation #2: Updated Oort constants based on Gaia and including radial motion

From step 4, we find the values used in further plotting and calculations; the results from step 3 are also mentioned in the main text of Section 3.3.

The most revelant values taken from step 4 are:

$\theta = 35 \pm 10\degree$ West of North

$v_* = 7.0 \pm 2.5$ km/s

Note that, as mentioned below, the exact re-estimated values may differ slightly within these uncertainties, but that does not affect the work and conclusions in the paper.

### 0) import the necessary packages

In [1]:
import numpy as np
from astropy.coordinates import ICRS, Galactic, SkyCoord, LSR
import astropy.units as u

### 1) Local ISM motion correction

The below function calculates the local motion at a given set of Galactic coordinates and distance, given the assumed Oort's constants and solar values $(U,V,W)_0$. This function is based on Comeron & Pasquali (2007), who refer to Scheffler & Elsasser (1987).

The standard values of $(U,V,W)_0$ are taken from Schonrich et al. (2010)

In [2]:
def local_motion(l, b, D, 
                 A = 12.5*u.km/u.s/u.kpc, B = -12.5*u.km/u.s/u.kpc, 
                 U = 11.1*u.km/u.s, V = 12.24*u.km/u.s, W = 7.25*u.km/u.s):

    C = 0.211 * u.mas / u.yr / (u.km / u.s / u.kpc)
    
    pm_l_cosb_0 = C*(A*np.cos(2.*l)*np.cos(b) + B*np.cos(b) 
                         + (U/D)*np.sin(l) - (V/D)*np.cos(l))
    pm_b_0 = C*(-1*A*np.sin(2.*l)*np.sin(b)*np.cos(b) 
                    + (U/D)*np.cos(l)*np.sin(b) + (V/D)*np.sin(l)*np.sin(b)
                    - (W/D)*np.cos(b))
    
    return (pm_l_cosb_0.to(u.mas / u.yr), pm_b_0.to(u.mas / u.yr))

### 2) The example calculation using pre-Gaia constraints

To demonstrate the calculation, we can first perform it using the parameters of LS 2355 as introduced in Sanchez Ayaso et al. (2018)

In [3]:
D_LS2355 = 2.1 * u.kpc # The pre-Gaia best distance; different from the paper value

In [4]:
# Transforming the proper motion from Sanchez Ayaso et al. (2018) from ICRS to Galactic coordinates:
LS2355_2018 = ICRS(ra=172.225759*u.degree, dec=-62.652733*u.degree,
                   pm_ra_cosdec=-7.8*u.mas/u.yr, pm_dec=3.5*u.mas/u.yr)
LS2355_2018_Gal = LS2355_2018.transform_to(Galactic()) 
LS2355_2018_Gal

<Galactic Coordinate: (l, b) in deg
    (293.61336075, -1.28375861)
 (pm_l_cosb, pm_b) in mas / yr
    (-8.50422629, 0.87643324)>

In [5]:
# Calculating the local motion at the position and distance of LS 2355:
(pm_l_cosb_0, pm_b_0) = local_motion(l = LS2355_2018_Gal.l.degree*u.degree, b = LS2355_2018_Gal.b.degree*u.degree,
                                     D = D_LS2355)

In [7]:
# Defining a new astropy object with the same Galactic position but the corrected Galactic motion:
LS2355_corrected = Galactic(l=LS2355_2018_Gal.l.degree*u.degree, b=LS2355_2018_Gal.b.degree*u.degree,
                   pm_l_cosb=LS2355_2018_Gal.pm_l_cosb - pm_l_cosb_0, pm_b=LS2355_2018_Gal.pm_b - pm_b_0)

# Converting to ICRS:
LS2355_corrected_ICRS = LS2355_corrected.transform_to(ICRS()) 
LS2355_corrected_ICRS

<ICRS Coordinate: (ra, dec) in deg
    (172.225759, -62.652733)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-1.92062662, 2.35417436)>

In [8]:
# Calculating the angle
theta = np.arctan(LS2355_corrected_ICRS.pm_ra_cosdec / LS2355_corrected_ICRS.pm_dec)
theta.to(u.degree)

<Quantity -39.20889793 deg>

In [9]:
# Calculating the velocity:
((((LS2355_corrected_ICRS.pm_ra_cosdec / (1.*u.radian)) * D_LS2355)**2 +
((LS2355_corrected_ICRS.pm_dec / (1.*u.radian)) * D_LS2355)**2)**0.5).to(u.km/u.second)

<Quantity 30.24571021 km / s>

### 3) The Gaia-based calculation #1: Oort constants from Comeron & Pasquali (2007) and no radial motion

We then demonstrate the calculation using the Gaia constraints; we here do not yet include radial motion or updated Oort constants (based on Gaia). We will include both in step 4.

However, we will explicitly introduce uncertainties in the calculation after first repeating the steps from part 2.

In [16]:
D_LS2355 = 2.2 * u.kpc # The Gaia-based distance

In [17]:
# Converting the Galactic coordinates, with the Gaia proper motion:
LS2355 = ICRS(ra=172.225759*u.degree, dec=-62.652733*u.degree,
                   pm_ra_cosdec=-6.40922*u.mas/u.yr, pm_dec=1.678*u.mas/u.yr)
LS2355_Gal = LS2355.transform_to(Galactic()) 
LS2355_Gal

<Galactic Coordinate: (l, b) in deg
    (293.61336075, -1.28375861)
 (pm_l_cosb, pm_b) in mas / yr
    (-6.61208326, -0.4173008)>

In [18]:
# Calculating the local motion for the Comeron & Pasquali (2007) Oort's constants:
(pm_l_cosb_0, pm_b_0) = local_motion(l = LS2355_Gal.l.degree*u.degree, b = LS2355_Gal.b.degree*u.degree,
                                     D = D_LS2355)

In [19]:
# Correcting the motion and converting back from Galactic coordinates
LS2355_corrected = Galactic(l=LS2355_Gal.l.degree*u.degree, b=LS2355_Gal.b.degree*u.degree,
                   pm_l_cosb=LS2355_Gal.pm_l_cosb - pm_l_cosb_0, pm_b=LS2355_Gal.pm_b - pm_b_0)
LS2355_corrected_ICRS = LS2355_corrected.transform_to(ICRS()) 
LS2355_corrected_ICRS

<ICRS Coordinate: (ra, dec) in deg
    (172.225759, -62.652733)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-0.60538092, 0.52299588)>

In [20]:
# Calculating angle and velocity:
theta = np.arctan(LS2355_corrected_ICRS.pm_ra_cosdec / LS2355_corrected_ICRS.pm_dec)
theta.to(u.degree)

<Quantity -49.17586964 deg>

In [21]:
((((LS2355_corrected_ICRS.pm_ra_cosdec / (1.*u.radian)) * D_LS2355)**2 +
((LS2355_corrected_ICRS.pm_dec / (1.*u.radian)) * D_LS2355)**2)**0.5).to(u.km/u.second)

<Quantity 8.34329807 km / s>

Then, we can perform these steps but repeat them $10^3$ times, varying the proper motions and Oort's constants to estimate the uncertainty on direction and velocity:

In [23]:
# Gaia values:
pm_ra_cosdec0 = -6.40922*u.mas/u.yr
pm_dec0 = 1.678*u.mas/u.yr
error = 0.02*u.mas/u.yr

# Repeating the steps above 10^3 times:
v_sim = []
theta_sim = []
for i in range(1000):

    # Varying the Gaia proper motions within their errors:
    pm_ra_cosdec_i = np.random.normal(loc=pm_ra_cosdec0.value, scale=error.value, size=1)[0]*u.mas/u.yr
    pm_dec_i = np.random.normal(loc=pm_dec0.value, scale=error.value, size=1)[0]*u.mas/u.yr

    # Converting to Galactic coordinates:
    LS2355 = ICRS(ra=172.225759*u.degree, dec=-62.652733*u.degree,
                       pm_ra_cosdec=pm_ra_cosdec_i, pm_dec=pm_dec_i)
    LS2355_Gal = LS2355.transform_to(Galactic()) 

    # Varying the Oort's constants; assuming sigma = 1 km/s/kpc to estimate the effect. Note that A+B remains constant. 
    dOort = np.random.normal(loc=0, scale=1, size=1)[0] * u.km/u.s/u.kpc
    Ai = 12.5*u.km/u.s/u.kpc + dOort
    Bi = -12.5*u.km/u.s/u.kpc - dOort

    # Calculating the local correction for these Oort constants:
    (pm_l_cosb_0, pm_b_0) = local_motion(l = LS2355_Gal.l.degree*u.degree, b = LS2355_Gal.b.degree*u.degree,
                                         D = D_LS2355, A=Ai, B=Bi)

    # Correcting the motion and transforming back:
    LS2355_corrected = Galactic(l=LS2355_Gal.l.degree*u.degree, b=LS2355_Gal.b.degree*u.degree,
                       pm_l_cosb=LS2355_Gal.pm_l_cosb - pm_l_cosb_0, pm_b=LS2355_Gal.pm_b - pm_b_0)
    LS2355_corrected_ICRS = LS2355_corrected.transform_to(ICRS()) 

    # Calculating and saving the angle and velocity:
    theta = np.arctan(LS2355_corrected_ICRS.pm_ra_cosdec / LS2355_corrected_ICRS.pm_dec)
    theta_sim.append(theta.to(u.degree).value)

    v = ((((LS2355_corrected_ICRS.pm_ra_cosdec / (1.*u.radian)) * D_LS2355)**2 +
    ((LS2355_corrected_ICRS.pm_dec / (1.*u.radian)) * D_LS2355)**2)**0.5).to(u.km/u.second)
    v_sim.append(v.value)

In [24]:
# Turning velocity and angle into arrays with correct units:
v_sim = np.asarray(v_sim) * u.km/u.second
theta_sim = np.asarray(theta_sim) * u.degree

In [27]:
# Printing the velocity and error: (note: error given for this assumed uncertainty on A and B)
np.mean(v_sim), np.std(v_sim)

(<Quantity 8.72818663 km / s>, <Quantity 3.14444989 km / s>)

In [28]:
# Printing the angle and error: (note: error given for this assumed uncertainty on A and B)
np.mean(theta_sim), np.std(theta_sim)

(<Quantity -45.00968367 deg>, <Quantity 16.01226913 deg>)

### 4) The Gaia-based calculation #2: Updated Oort constants based on Gaia and including radial motion

Finally, the values used in the remainder of the paper: using updated Oort's constants from  https://ui.adsabs.harvard.edu/abs/2017MNRAS.468L..63B/abstract). We also include radial motion.

In [30]:
# Calculating what the radial velocity corresponds to in the ICRS coordinates, for the position considered here:
# This calculation corrects from the LSR value, as reported in Sanchez Ayaso et al. (2018) to the Galactic/ICRS value (which are the same): 
# While we provide all 6 parameters, only the radial velocity and position are relevant here:
lsr = LSR(ra=172.225759*u.degree, dec=-62.652733*u.degree,
          distance=D_LS2355, 
          radial_velocity=-3.0*u.km/u.s,
          pm_ra_cosdec=-6.40922*u.mas/u.yr, pm_dec=1.678*u.mas/u.yr)
v_r = lsr.transform_to(ICRS()).radial_velocity

# Showing the corrected radial velocity:
v_r

<Quantity 3.92962034 km / s>

In [31]:
# Repeating the earlier calculation; including radial velocity and updated Oort's constants:

# Gaia values:
pm_ra_cosdec0 = -6.40922*u.mas/u.yr
pm_dec0 = 1.678*u.mas/u.yr
error = 0.02*u.mas/u.yr

# Error on radial velocity:
dv_r = 4.5 * u.km/u.second

# Repeating the steps above 10^3 times:
v_sim = []
v0_sim = []
theta_sim = []
pm_ra_cosdec_sim = []
pm_dec_sim = []
for i in range(1000):

    # Varying the Gaia proper motions within their errors:
    pm_ra_cosdec_i = np.random.normal(loc=pm_ra_cosdec0.value, scale=error.value, size=1)[0]*u.mas/u.yr
    pm_dec_i = np.random.normal(loc=pm_dec0.value, scale=error.value, size=1)[0]*u.mas/u.yr

    # Converting to Galactic coordinates:
    LS2355 = ICRS(ra=172.225759*u.degree, dec=-62.652733*u.degree,
                       pm_ra_cosdec=pm_ra_cosdec_i, pm_dec=pm_dec_i)
    LS2355_Gal = LS2355.transform_to(Galactic()) 

    # Calculating a random radial velocity from its mean and error:
    v_r_i = np.random.normal(loc=v_r.value, scale=dv_r.value, size=1)[0]*u.km/u.second

    # Variation of the updated Oort's constants:
    dA = np.random.normal(loc=0, scale=0.4, size=1)[0] * u.km/u.s/u.kpc
    Ai = 15.3*u.km/u.s/u.kpc + dA
    Bi = -11.9*u.km/u.s/u.kpc - dA

    # Calculating the local correction for these Oort constants:
    (pm_l_cosb_0, pm_b_0) = local_motion(l = LS2355_Gal.l.degree*u.degree, b = LS2355_Gal.b.degree*u.degree,
                                         D = D_LS2355, A=Ai, B=Bi)

    # Correcting the motion and transforming back:
    LS2355_corrected = Galactic(l=LS2355_Gal.l.degree*u.degree, b=LS2355_Gal.b.degree*u.degree,
                       pm_l_cosb=LS2355_Gal.pm_l_cosb - pm_l_cosb_0, pm_b=LS2355_Gal.pm_b - pm_b_0)
    LS2355_corrected_ICRS = LS2355_corrected.transform_to(ICRS()) 

    # Saving the updated/corrected proper motions, angles, and velocities.
    pm_ra_cosdec_sim.append(LS2355_corrected_ICRS.pm_ra_cosdec.value)
    pm_dec_sim.append(LS2355_corrected_ICRS.pm_dec.value)
    
    theta = np.arctan(LS2355_corrected_ICRS.pm_ra_cosdec / LS2355_corrected_ICRS.pm_dec)
    theta_sim.append(theta.to(u.degree).value)

    # Note: we calculate v and v0, where for v0, we don't use the variation in v_r, but only in the proper motion in tangetial direction.
    v = ((((LS2355_corrected_ICRS.pm_ra_cosdec / (1.*u.radian)) * D_LS2355)**2 +
    ((LS2355_corrected_ICRS.pm_dec / (1.*u.radian)) * D_LS2355)**2 + v_r_i**2)**0.5).to(u.km/u.second)
    
    v0 = ((((LS2355_corrected_ICRS.pm_ra_cosdec / (1.*u.radian)) * D_LS2355)**2 +
    ((LS2355_corrected_ICRS.pm_dec / (1.*u.radian)) * D_LS2355)**2 + v_r**2)**0.5).to(u.km/u.second)
    
    v_sim.append(v.value)
    v0_sim.append(v0.value)

In [37]:
# The proper motion in RA:
np.std(pm_ra_cosdec_sim), np.mean(pm_ra_cosdec_sim)

(0.14138846992288567, -0.3423069197866604)

In [38]:
# The proper motion in declination:
np.mean(pm_dec_sim), np.std(pm_dec_sim)

(0.44556546555815524, 0.04908364507528602)

In [39]:
v_sim = np.asarray(v_sim) * u.km/u.second
v0_sim = np.asarray(v0_sim) * u.km/u.second
theta_sim = np.asarray(theta_sim) * u.degree

In [44]:
# Velocity and error: we use the mean of v0_sim as v_sim is heavily skewed. 
# Note that exact values of v and angle may vary slightly between different executions of the 1000 calculations. 
# These variations lie within the uncertainty and do not affect the conclusions or inferences of the paper.
np.mean(v0_sim), np.std(v_sim)

(<Quantity 7.1520503 km / s>, <Quantity 2.4516716 km / s>)

In [41]:
# Angle and error:
np.mean(theta_sim), np.std(theta_sim)

(<Quantity -35.60863959 deg>, <Quantity 10.16105355 deg>)